<a href="https://colab.research.google.com/github/mkkadambi/machine-learning/blob/main/NeuralNetworks_MNIST_DigitRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementing Neural Networks from scratch without using Tensorflow
<br><br>
The MNIST data set consists of 70,000 images of handwritten digits. It consists of digits from 0 to 9, the requirement is to classify the class to which the image belongs. The images in the MNIST data set are 28X28 pixels, and the input layer shall have 784 neurons (each neuron takes 1 pixel as the input). The output layer shall have 10 neurons, with each giving the probability of the input image belonging to any of the 10 classes. The image is classified into the class that is represented by the neuron with the highest probability. In the diagram below, you can see a few sample images that are available in the dataset.
<br><br>


<img src="https://images.upgrad.com/80bd8b72-de1d-4ea6-9923-6b9fc3d03ed9-8.png">


In [1]:
# Import required libraries:
# For model file handling
import pickle
import gzip

import numpy as np
import pandas as pd
import h5py

import sklearn
import sklearn.datasets
import scipy

# For plotting and image displays
import matplotlib.pyplot as plt
from PIL import Image
from scipy import ndimage
%matplotlib inline

# Download Dataset

In [7]:
data_url = "https://cdn.upgrad.com/uploads/production/2f798b87-a95d-4d53-b34b-166f291c022b/mnist.pkl.gz"
# Download the dataset to the local path
!wget $data_url
!ls

--2022-06-16 01:23:51--  https://cdn.upgrad.com/uploads/production/2f798b87-a95d-4d53-b34b-166f291c022b/mnist.pkl.gz
Resolving cdn.upgrad.com (cdn.upgrad.com)... 108.138.94.76, 108.138.94.107, 108.138.94.27, ...
Connecting to cdn.upgrad.com (cdn.upgrad.com)|108.138.94.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17051982 (16M) [application/x-gzip]
Saving to: ‘mnist.pkl.gz’

mnist.pkl.gz        100%[===================>]  16.26M  8.23MB/s    in 2.0s    

2022-06-16 01:23:54 (8.23 MB/s) - ‘mnist.pkl.gz’ saved [17051982/17051982]

mnist.pkl.gz  sample_data


# Load Dataset
* uncompress the dataset file
* read the file into python variables

In [10]:
def load_data(file='mnist.pkl.gz'):
  '''
  Function to uncompress .gz file into a .pkl file. Load the .pkl file as 
  train_data, validation_data and test_data in that order
  file: string, .gz filename containing .pkl dataset file
  '''
  # Open the file in read-only binary mode using gzip library
  file_pointer = gzip.open(file, mode='rb')
  # point the file_pointer to the start of the file
  file_pointer.seek(0)
  # load the three dataset objects
  # The data was created using latin1 encoding
  train_data, validation_data, test_data = pickle.load(file_pointer,
                                                       encoding='latin1')
  return train_data, validation_data, test_data

In [14]:
train, val, test = load_data()
print(f'train data type = {type(train)}, len = {len(train)}')
print(f'val data type = {type(val)}, len = {len(val)}')
print(f'test data type = {type(test)}, len = {len(test)}')

train data type = <class 'tuple'>, len = 2
val data type = <class 'tuple'>, len = 2
test data type = <class 'tuple'>, len = 2
